<a href="https://colab.research.google.com/github/sobti/TSAI/blob/master/Bert_IMDB/Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers


     |████████████████████████████████| 1.9MB 20.1MB/s 
     |████████████████████████████████| 890kB 56.3MB/s 
     |████████████████████████████████| 3.2MB 54.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=6fb71a88f311af587ffeb144849c2ff004de75b7b52a8c0bf1e8744f444f54c8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from transformers import BertTokenizer

In [3]:
input= 'What is your name?'
input1 = 'My name is punit.'
tokenz=BertTokenizer.from_pretrained('bert-base-uncased')

In [12]:
token=tokenz.encode(input,return_tensors = 'pt',max_length=64)
token
#token1=tokenz.encode(input1)
#tokenz.encode()

tensor([[ 101, 2054, 2003, 2115, 2171, 1029,  102]])

In [16]:
inp=tokenz(input,padding=True,max_length=10)
inp

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [13]:
from keras.preprocessing.sequence import pad_sequences
input_ids = pad_sequences(token, maxlen=64, dtype="long", 
                          value=0, truncating="post", padding="post")

In [44]:
att_mask=[]
input_ids[0][1].item()

2054

In [46]:
for i in range(input_ids.shape[1]):
  if input_ids[0][i].item() == 0:
   att_mask.append(0)
  else :
   att_mask.append(1)

In [51]:
from google.colab import drive
import os
import sys
drive.mount('/content/gdrive/')
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/')
os.chdir('/content/gdrive/My Drive/Colab Notebooks')

Mounted at /content/gdrive/


In [54]:
!7z x '/content/gdrive/My Drive/Colab Notebooks/IMDB Dataset.csv.zip' -o'/content/data/'


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/gdrive/My Drive/Colab Notebooks/                                                   1 file, 26962657 bytes (26 MiB)

Extracting archive: /content/gdrive/My Drive/Colab Notebooks/IMDB Dataset.csv.zip
--
Path = /content/gdrive/My Drive/Colab Notebooks/IMDB Dataset.csv.zip
Type = zip
Physical Size = 26962657

  0%     28% - IMDB Dataset.csv                        55% - IMDB Dataset.csv                        84% - IMDB Dataset.csv                       Everything is Ok

Size:       66212309
Compressed

In [58]:
from sklearn.model_selection import train_test_split as t

In [59]:
import pandas as pd

# New section

In [60]:
data=pd.read_csv('/content/data/IMDB Dataset.csv')

In [67]:
data['sentiment'][1]

'positive'

In [72]:
k=1
labels=[]
for i in range(len(data)):
  if data['sentiment'][i] == 'positive':
    labels.append(1)
    k=k+1
  else:
     labels.append(0)
     k=k+1

In [77]:
#X_train, X_test, y_train, y_test = train_test_split(data['review'],labels, test_size=0.33, random_state=42)

In [82]:
BERTMODEL = "bert-base-uncased"

tokenizer = BertTokenizer.from_pretrained(BERTMODEL,
                                          do_lower_case=True)

In [165]:
k=1
sen=[]
#labels=[]
for i in range(len(data)):
  sen.append(tokenizer.encode(data['review'][i],add_special_tokens = True,max_length = 128)) 
  
  
  

In [168]:
print('Max sentence length: ', max([len(se) for se in sen]))


Max sentence length:  128


In [169]:
from keras.preprocessing.sequence import pad_sequences
input = pad_sequences(sen, maxlen=128, truncating="post", padding="post",dtype='long',value=0)

# New section

In [172]:
# Create attention masks
attention_masks = []
# For each sentence...
for sent in input:
    
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

In [175]:
# Use train_test_split to split our data into train and validation sets for
# training
from sklearn.model_selection import train_test_split
# Use 90% for training and 10% for validation.
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input, labels, 
                                                            random_state=2018, test_size=0.1)
# Do the same for the masks.
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=2018, test_size=0.1)

In [177]:
import torch
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [183]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size = 32
# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [185]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [186]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
import torch.optim as optim
optimizer = optim.AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
from transformers import get_linear_schedule_with_warmup
# Number of training epochs (authors recommend between 2 and 4)
epochs = 4
# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [188]:
import numpy as np
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
import time
import datetime
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))    

In [190]:
import random
device='cuda'
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
# Store the average loss after each epoch so we can plot them.
loss_values = []
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    # Reset the total loss for this epoch.
    total_loss = 0
    model.train()
    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
     
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        model.zero_grad()        
  
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        
        loss = outputs[0]
 
        total_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
      
        optimizer.step()
        # Update the learning rate.
        scheduler.step()
    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy
        # Track the number of batches
        nb_eval_steps += 1
    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
print("")
print("Training complete!")     


======== Epoch 1 / 4 ========
Training...
  Batch    40  of  1,407.    Elapsed: 0:00:26.
  Batch    80  of  1,407.    Elapsed: 0:00:53.
  Batch   120  of  1,407.    Elapsed: 0:01:19.
  Batch   160  of  1,407.    Elapsed: 0:01:47.
  Batch   200  of  1,407.    Elapsed: 0:02:14.
  Batch   240  of  1,407.    Elapsed: 0:02:42.
  Batch   280  of  1,407.    Elapsed: 0:03:10.
  Batch   320  of  1,407.    Elapsed: 0:03:39.
  Batch   360  of  1,407.    Elapsed: 0:04:07.
  Batch   400  of  1,407.    Elapsed: 0:04:36.
  Batch   440  of  1,407.    Elapsed: 0:05:04.
  Batch   480  of  1,407.    Elapsed: 0:05:32.
  Batch   520  of  1,407.    Elapsed: 0:06:01.
  Batch   560  of  1,407.    Elapsed: 0:06:29.
  Batch   600  of  1,407.    Elapsed: 0:06:58.
  Batch   640  of  1,407.    Elapsed: 0:07:26.
  Batch   680  of  1,407.    Elapsed: 0:07:55.
  Batch   720  of  1,407.    Elapsed: 0:08:23.
  Batch   760  of  1,407.    Elapsed: 0:08:51.
  Batch   800  of  1,407.    Elapsed: 0:09:20.
  Batch   840  of